In [4]:
import os
from hipecta.data import PCalibRun
from hipecta.data import PSimulation
from hipecta.data import ctaTelescope2Matrix
from hipecta import core
import numpy as np
import h5py

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
%config InlineBackend.figure_format = 'retina'
np.set_printoptions(precision=3)
from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

## First try of pcalibrun/psimu to hdf5 converter

for a single tuple (pcalibrun,psimu)

In [86]:
pruncalibfilename = ['/Users/mikael/projets_CTA/gamma_20deg_0deg_run110___cta-prod3_desert-2150m-Paranal-merged.pcalibRun']
simufilename = ['/Users/mikael/projets_CTA/gamma_20deg_0deg_run110___cta-prod3_desert-2150m-Paranal-merged.psimu']
hdf5filename = '/Users/mikael/projets_CTA/gamma.hdf5'

In [87]:
pr = PCalibRun()
pr.load(pruncalibfilename)
ps = PSimulation()
ps.load(simufilename)

In [6]:
telescopeTypeDict = {0:'DRAGON', 1:'NECTAR', 2:'FLASH', 3:'SCT', 4:'ASTRI', 5:'DC', 6:'GCT'}
for telType in telescopeTypeDict:
    print(telType)

0
1
2
3
4
5
6


In [88]:
hdf5StuctureDict = {}

if os.path.isfile('/Users/mikael/projets_CTA/gamma.hdf5'):
  gammaHdf = h5py.File(hdf5filename,'a')
  # Load hdf5 file groups and datasets
  
else:
  gammaHdf = h5py.File(hdf5filename,'w')
  # Create hdf5 structure with empty datasets
  # Telescope data
  for telType in telescopeTypeDict:
    telGrp = {}
    telGrp["group"] = gammaHdf.create_group(telescopeTypeDict[telType])
    telGrp["showerId"] = telGrp["group"].creat_dataset("showerId",dtype=np.uint64)
    telGrp["images"] = telGrp["group"].creat_dataset("images",dtype=np.float32)
    telGrp["pixelsPosition"] = telGrp["group"].creat_dataset("pixelsPosition",dtype=np.float32)
    telGrp["eventId"] = telGrp["group"].creat_dataset("eventId",dtype=np.uint64)
    telGrp["telescopeId"] = telGrp["group"].creat_dataset("telescopeId",dtype=np.uint64)
    telGrp["injTable"] = telGrp["group"].creat_dataset("injTable",dtype=np.uint64)
    telGrp["nbRow"] = telGrp["group"].creat_dataset("nbRow",dtype=np.int8)
    telGrp["nbCol"] = telGrp["group"].creat_dataset("nbCol",dtype=np.int8)
    hdf5StuctureDict[telType] = telGrp
  # Shower simulation data
  showerSimuGrp = gammaHdf.create_group("showerSimu")
  showerDataAltitude = showerSimuGrp.create_dataset('altitude',dtype=np.float32)
  showerDataAzimuth = showerSimuGrp.create_dataset('azimuth',dtype=np.float32)
  showerDataCmax = showerSimuGrp.create_dataset('cmax',dtype=np.float32)
  showerDataDepthStart = showerSimuGrp.create_dataset('depthStart',dtype=np.float32)
  showerDataEmax = showerSimuGrp.create_dataset('emax',dtype=np.float32)
  showerDataEnergy = showerSimuGrp.create_dataset('energy',dtype=np.float32)
  showerDataHeight = showerSimuGrp.create_dataset('heightFirstInteraction',dtype=np.float32)
  showerDataHmax = showerSimuGrp.create_dataset('hmax',dtype=np.float32)
  showerDataShowerId = showerSimuGrp.create_dataset('showerId',dtype=np.uint64)
  showerDataParticleType = showerSimuGrp.create_dataset('particleType',dtype=np.int32)
  showerDataXmax = showerSimuGrp.create_dataset('xmax',dtype=np.float32)
  # Event simulation data
  eventSimuGrp = gammaHdf.create_group("eventSimu")
  eventDataEventId = eventSimuGrp.create_dataset('eventId',dtype=np.uint64)
  eventDataShowerId = eventSimuGrp.create_dataset('showerId',dtype=np.uint64)
  eventDataXCore = eventSimuGrp.create_dataset('xCore',dtype=np.float32)
  eventDataYCore = eventSimuGrp.create_dataset('yCore',dtype=np.float32)
  # Telescope Infos
  telInfos = gammaHdf.create_group("telescopeInfos")
  telescopeInfoId = telInfos.create_dataset('telescopeId',dtype=np.uint64)
  telescopeInfoPosition = telInfos.create_dataset('telescopePosition',dtype=np.float32)
  telescopeInfoFocal = telInfos.create_dataset('telescopeFocal',dtype=np.float32)

### Fetching data

In [90]:
# Shower simulation data
altitude = np.array([sh.altitude for sh in ps.tabSimuShower])
azimuth = np.array([sh.azimuth for sh in ps.tabSimuShower])
cmax = np.array([sh.cmax for sh in ps.tabSimuShower])
depthStart = np.array([sh.depthStart for sh in ps.tabSimuShower])
emax = np.array([sh.emax for sh in ps.tabSimuShower])
energy = np.array([sh.energy for sh in ps.tabSimuShower])
heightFirstInteraction = np.array([sh.heightFirstInteraction for sh in ps.tabSimuShower])
hmax = np.array([sh.hmax for sh in ps.tabSimuShower])
showerId = np.array([sh.id for sh in ps.tabSimuShower])
particleType = np.array([sh.particleType for sh in ps.tabSimuShower])
xmax = np.array([sh.xmax for sh in ps.tabSimuShower])

# Event simulation data
eventIdSim = np.array([ev.id for ev in ps.tabSimuEvent])
showerIdSim = np.array([ev.showerNum for ev in ps.tabSimuEvent])
xCore = np.array([ev.xCore for ev in ps.tabSimuEvent])
yCore = np.array([ev.yCore for ev in ps.tabSimuEvent])

# Telescope infos
telId = np.array([tel.telescopeId for tel in pr.tabTelescope])
telPosition = np.array([posTel for posTel in pr.header.tabPosTel])
telFocal = np.array([focalTel for focalTel in pr.header.tabFocalTel])

In [91]:
# Write to hdf5 as group showerSimu
showerSimuGrp = gammaHdf.create_group("showerSimu")
ds1 = showerSimuGrp.create_dataset('altitude',data=altitude,dtype=np.float32)
ds2 = showerSimuGrp.create_dataset('azimuth',data=azimuth,dtype=np.float32)
ds3 = showerSimuGrp.create_dataset('cmax',data=cmax,dtype=np.float32)
ds4 = showerSimuGrp.create_dataset('depthStart',data=depthStart,dtype=np.float32)
ds5 = showerSimuGrp.create_dataset('emax',data=emax,dtype=np.float32)
ds6 = showerSimuGrp.create_dataset('energy',data=energy,dtype=np.float32)
ds7 = showerSimuGrp.create_dataset('heightFirstInteraction',data=heightFirstInteraction,dtype=np.float32)
ds8 = showerSimuGrp.create_dataset('hmax',data=hmax,dtype=np.float32)
ds9 = showerSimuGrp.create_dataset('showerId',data=showerId,dtype=np.uint64)
ds10 = showerSimuGrp.create_dataset('particleType',data=particleType,dtype=np.int32)
ds11 = showerSimuGrp.create_dataset('xmax',data=xmax,dtype=np.float32)

# Write to hdf5 as group eventSimu
eventSimuGrp = gammaHdf.create_group("eventSimu")
ds12 = eventSimuGrp.create_dataset('eventId', data=eventIdSim,dtype=np.uint64)
ds13 = eventSimuGrp.create_dataset('showerId', data=showerIdSim,dtype=np.uint64)
ds14 = eventSimuGrp.create_dataset('xCore', data=xCore,dtype=np.float32)
ds15 = eventSimuGrp.create_dataset('yCore', data=yCore,dtype=np.float32)

# Write to hdf5 as group telescopeInfos
telInfos = gammaHdf.create_group("telescopeInfos")
ds16 = telInfos.create_dataset('telescopeId', data=telId,dtype=np.uint64)
ds17 = telInfos.create_dataset('telescopePosition', data=telPosition,dtype=np.float32)
ds18 = telInfos.create_dataset('telescopeFocal', data=telFocal,dtype=np.float32)

In [92]:
# Telescope data
telescopeType = set([tel.telescopeType for tel in pr.tabTelescope])
print(telescopeType)
telDict = {}
datasets = []
for telType in telescopeType:
  # Create telescope type group in hdf5
  telDict[telType] = gammaHdf.create_group(telescopeTypeDict[telType])
  # Telescope event data
  images = np.array([event.tabPixel for tel in pr.tabTelescope if tel.telescopeType == telType for event in tel.tabTelEvent])
  eventId = np.array([event.eventId for tel in pr.tabTelescope if tel.telescopeType == telType for event in tel.tabTelEvent])
  telescopeId =  np.array([tel.telescopeId for tel in pr.tabTelescope if tel.telescopeType == telType for event in tel.tabTelEvent])
  pixelsPosition = [tel.tabPos.tabPixelPosXY for tel in pr.tabTelescope if tel.telescopeType == telType][0]
  # Add matrix form images
  injTable, nbRow, nbCol = core.createAutoInjunctionTable(pixelsPosition)
  # Add shower id
  showerIdData = np.array([showerIdSim[np.where(eventIdSim==ev)] for ev in eventId])
  
  # Add datasets to group
  datasets.append(telDict[telType].create_dataset('images', data=images,dtype=np.float32))
  datasets.append(telDict[telType].create_dataset('eventId', data=eventId,dtype=np.uint64))
  datasets.append(telDict[telType].create_dataset('showerId', data=showerIdData,dtype=np.uint64))
  datasets.append(telDict[telType].create_dataset('telescopeId', data=telescopeId,dtype=np.uint64))
  datasets.append(telDict[telType].create_dataset('pixelsPosition', data=pixelsPosition,dtype=np.float32))
  datasets.append(telDict[telType].create_dataset('injTable', data=injTable,dtype=np.uint64))
  datasets.append(telDict[telType].create_dataset('nbRow', data=nbRow,dtype=np.int8))
  datasets.append(telDict[telType].create_dataset('nbCol', data=nbCol,dtype=np.int8))


{0, 1, 2, 4, 5, 6}


In [93]:
gammaHdf.close()